# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [64]:
# Use this cell to write your code for Task 1
import pandas as pd

# Load data
data = pd.read_csv("loyalty.csv")
clean_data = data.copy()

# Define replacement dictionary
replace_dict = {
    'spend': 0,
    'first_month': 0,
    'items_in_first_month': 0,
    'region': 'Unknown',
    'loyalty_years': '0-1',
    'joining_month': 'Unknown',
    'promotion': 'No'
}

# Handle missing values
clean_data.fillna(replace_dict, inplace=True)

# Clean numerical columns
clean_data["spend"] = clean_data["spend"].round(2)
clean_data["first_month"] = pd.to_numeric(clean_data["first_month"], errors='coerce').fillna(0).round(2)
clean_data["items_in_first_month"] = pd.to_numeric(clean_data["items_in_first_month"], errors='coerce').fillna(0).astype(int)

# Clean categorical columns
clean_data['region'] = (
    clean_data['region']
    .replace({'': 'Unknown'})
    .astype('category')
)

loyalty_order = ['0-1', '1-3', '3-5', '5-10', '10+']
clean_data['loyalty_years'] = pd.Categorical(
    clean_data['loyalty_years'],
    categories=loyalty_order,
    ordered=True
)

clean_data['joining_month'] = clean_data['joining_month'].astype('category')

clean_data['promotion'] = (
    clean_data['promotion']
    .str.upper()
    .map({'YES': 'Yes', 'NO': 'No'})
    .fillna('No')
    .astype('category')
)

# Verify
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   customer_id           1246 non-null   int64   
 1   spend                 1246 non-null   float64 
 2   first_month           1246 non-null   float64 
 3   items_in_first_month  1246 non-null   int64   
 4   region                1246 non-null   category
 5   loyalty_years         1246 non-null   category
 6   joining_month         1246 non-null   category
 7   promotion             1246 non-null   category
dtypes: category(4), float64(2), int64(2)
memory usage: 45.1 KB


# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [65]:
# Use this cell to write your code for Task 2
spend_by_years = (
    data
    .groupby('loyalty_years', as_index=False)
    .agg(
        avg_spend=('spend', lambda x: round(x.mean(), 2)),
        var_spend=('spend', lambda x: round(x.var(), 2))
    )
)
spend_by_years

,loyalty_years,avg_spend,var_spend
0,0-1,110.56,9.30
1,1-3,129.31,9.65
2,10+,117.41,16.72
3,3-5,124.55,11.09
4,5-10,135.15,14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [66]:
# Use this cell to write your code for Task 3
import pandas as pd
from sklearn.linear_model import LinearRegression

def clean(df):
    # Create a copy to avoid modifying the original dataframe
    cleaned_df = df.copy()
    
    # Dictionary for filling missing values
    fill_values = {
        'spend': 0,
        'first_month': 0,
        'items_in_first_month': 0,
        'region': 'Unknown',
        'loyalty_years': '0-1',
        'joining_month': 'Unknown',
        'promotion': 'No'
    }
    
    # Fill missing values
    cleaned_df.fillna(fill_values, inplace=True)
    
    # Handle numeric columns
    if 'spend' in cleaned_df.columns:
        cleaned_df['spend'] = pd.to_numeric(cleaned_df['spend'], errors='coerce').fillna(0).round(2)
    cleaned_df['first_month'] = pd.to_numeric(cleaned_df['first_month'], errors='coerce').fillna(0).round(2)
    cleaned_df['items_in_first_month'] = pd.to_numeric(cleaned_df['items_in_first_month'], \
                                                       errors='coerce').fillna(0).astype(int)
    
    # Clean categorical columns
    cleaned_df['region'] = (
        cleaned_df['region']
        .str.strip()
        .replace({'': 'Unknown'})
        .astype('category')
    )
    
    # Define and apply loyalty years ordering
    loyalty_order = ['0-1', '1-3', '3-5', '5-10', '10+']
    cleaned_df['loyalty_years'] = pd.Categorical(
        cleaned_df['loyalty_years'],
        categories=loyalty_order,
        ordered=True
    )
    
    # Clean joining month
    cleaned_df['joining_month'] = (
        cleaned_df['joining_month']
        .str.strip()
        .replace({'': 'Unknown'})
        .astype('category')
    )
    
    # Standardize promotion values
    cleaned_df['promotion'] = (
        cleaned_df['promotion']
        .str.upper()
        .str.strip()
        .map({'YES': 'Yes', 'NO': 'No', 'Y': 'Yes', 'N': 'No'})
        .fillna('No')
        .astype('category')
    )
    
    return cleaned_df

# Load the training and test data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Data cleaning (repeat same cleaning as Task 1)
train_clean = clean(train)
test_clean = clean(test)

# Prepare features and target
X_train = train_clean.drop(columns=['customer_id', 'spend'])
y_train = train_clean['spend']

# Convert categorical variables to dummy variables
X_train = pd.get_dummies(X_train, drop_first=True)

# Initialize and train baseline model
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Prepare test data (must use same features as training)
X_test = test_clean.drop(columns=['customer_id'])
X_test = pd.get_dummies(X_test, drop_first=True)

# Ensure test has same columns as train (fill missing with 0)
train_cols = set(X_train.columns)
test_cols = set(X_test.columns)

# Make predictions
predictions = baseline_model.predict(X_test)

# Create result dataframe
base_result = pd.DataFrame({
    'customer_id': test_clean['customer_id'],
    'spend': predictions.round(2)  # Round to 2 decimal places
})

# Verify output
print(base_result.head())

   customer_id   spend
0            5  140.70
1            7  148.73
2           16  140.81
3           17  150.65
4           19  153.63


# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [67]:
# Use this cell to write your code for Task 4
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load the training and test data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Data cleaning (repeat same cleaning as Task 1)
train_clean = clean(train)
test_clean = clean(test)

# Prepare features and target
X_train = train_clean.drop(columns=['customer_id', 'spend'])
y_train = train_clean['spend']

# Convert categorical variables to dummy variables
X_train = pd.get_dummies(X_train, drop_first=True)

# Initialize and train baseline model
baseline_model = RandomForestRegressor()
baseline_model.fit(X_train, y_train)

# Prepare test data (must use same features as training)
X_test = test_clean.drop(columns=['customer_id'])
X_test = pd.get_dummies(X_test, drop_first=True)

# Ensure test has same columns as train (fill missing with 0)
train_cols = set(X_train.columns)
test_cols = set(X_test.columns)

# Make predictions
predictions = baseline_model.predict(X_test)

# Create result dataframe
compare_result = pd.DataFrame({
    'customer_id': test_clean['customer_id'],
    'spend': predictions.round(2)  # Round to 2 decimal places
})

# Verify output
print(compare_result.head())

   customer_id   spend
0            5  140.56
1            7  148.86
2           16  140.96
3           17  150.86
4           19  153.66
